In [2]:
# We need to import these modules to get started
import numpy as np
import pandas as pd
import os
import platform
import matplotlib.pyplot as plt
%matplotlib inline

# This patch of code just ensures we get an easy to read font size for the duration of the notebook
import matplotlib
font = {'weight' : 'bold',
        'size'   : 15}
matplotlib.rc('font', **font)

In [3]:
# Set file location based on platform. 
platstring = platform.platform()
if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/Brain2023/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on Code Ocean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2023/"

In [4]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

manifest_file = os.path.join(data_root,'allen-brain-observatory/visual-coding-2p/manifest.json')

# Create data cache object 
boc = BrainObservatoryCache(manifest_file=manifest_file)

In [5]:
# Load in data for session with the most neurons

In [6]:
slc17a7_sessions = pd.read_csv("possible_sessions_to_study.csv")

# Select the relevant data for the session with the most neurons
desired_container_id = slc17a7_sessions.experiment_container_id[np.where(slc17a7_sessions.number_cells==max(slc17a7_sessions.number_cells))[0][0]]
desired_container = boc.get_ophys_experiments(experiment_container_ids=[desired_container_id])
desired_container = sorted(desired_container, key=lambda x: x['session_type']) # sort based on session type so A comes first

# Create a list of three arrays that contains cell IDs for each session
cells_in_each_session = []
i=0
for session in desired_container:
    session_id = session["id"]
    session_data = boc.get_ophys_experiment_data(ophys_experiment_id=session_id)
    specimen_ids = session_data.get_cell_specimen_ids()
    cells_in_each_session.append(specimen_ids)
    print(f"Session {i+1} has {specimen_ids.shape[0]} cells")
    i+=1

Session 1 has 475 cells
Session 2 has 347 cells
Session 3 has 428 cells


In [8]:
# isolate the cell IDs for only the overlapping sessions
cells1 = list(cells_in_each_session[0])
cells2 = list(cells_in_each_session[1])
cells3 = list(cells_in_each_session[2])

overlapping_cell_ids = [cell for cell in cells1 if cell in cells2 and cell in cells3]
np.savetxt("overlapping_cell_ids.txt", overlapping_cell_ids)

In [ ]:
# for each session, extract cell indices 

In [9]:
# Get session IDs for each session
session_one_id = desired_container[0]["id"]
session_two_id = desired_container[1]["id"]
session_three_id = desired_container[2]["id"]

# load in session data for each session
session_one = boc.get_ophys_experiment_data(ophys_experiment_id=session_one_id)
session_two = boc.get_ophys_experiment_data(ophys_experiment_id=session_two_id)
session_three = boc.get_ophys_experiment_data(ophys_experiment_id=session_three_id)

# Get indices for overlapping cells in each session - isolates analysis to only overlapping cells
cell_indices_one = session_one.get_cell_specimen_indices(cell_specimen_ids=overlapping_cell_ids)
cell_indices_two = session_two.get_cell_specimen_indices(cell_specimen_ids=overlapping_cell_ids)
cell_indices_three = session_three.get_cell_specimen_indices(cell_specimen_ids=overlapping_cell_ids)